In [2]:
! pip install -q auto-gptq optimum

In [3]:
LLAMA2_path = 'meta-llama/llama-2-7b-chat-hf'
LLAMA3_path = 'meta-llama/Meta-Llama-3-8B-Instruct'

In [4]:
# import transformers
import torch

def load_model(path, tokenizer=None):
    quantization_config = transformers.GPTQConfig(bits=4, dataset = "c4", tokenizer=tokenizer)
    model = transformers.AutoModelForCausalLM.from_pretrained(path, attn_implementation="flash_attention_2",
                                                 device_map="auto", quantization_config=quantization_config, trust_remote_code=True)
    #model = transformers.AutoModelForCausalLM.from_pretrained(path, attn_implementation="flash_attention_2", device_map="auto",
    #                                            trust_remote_code=True, torch_dtype=torch.bfloat16)
    return model

In [5]:
from numpy import random

def generate_prompt_landmark(n_garbage, seed, percent):
    """Generates a text file and inserts an passkey at a random position."""
    rnd_state = random.get_state()
    random.seed(seed)
    # n_garbage_prefix = random.randint(0, n_garbage)
    n_garbage_prefix = int(percent * n_garbage)
    n_garbage_suffix = n_garbage - n_garbage_prefix

    task_description = "There is an important info hidden inside a lot of irrelevant text. Find it and memorize them. I will quiz you about the important information there."
    garbage = "The grass is green. The sky is blue. The sun is yellow. Here we go. There and back again."
    garbage_inf = " ".join([garbage] * 50000)
    assert len(garbage_inf) >= n_garbage
    garbage_prefix = garbage_inf[:n_garbage_prefix]
    garbage_suffix = garbage_inf[:n_garbage_suffix]
    pass_key = random.randint(50000, 500000)

    information_line = f"The pass key is {pass_key}. Remember it. {pass_key} is the pass key."
    final_question = "What is the pass key? The pass key is"
    lines = [
        task_description,
        garbage_prefix,
        information_line,
        garbage_suffix,
        final_question,
    ]
    random.set_state(rnd_state)
    return "\n".join(lines), str(pass_key)

In [6]:
import transformers

In [7]:
def tokenize(tokenizer, prompt):
    return tokenizer(prompt, return_tensors="pt").input_ids

In [8]:
def generate(model, tokens, max_tokens):

    generation_output = model.generate(
        input_ids=tokens.to(model.device), max_new_tokens=max_tokens, num_beams=1, use_cache=False
    )
    return generation_output

In [9]:
import os
import sys
# sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))


In [10]:
from chunkllama_attn_replace import replace_with_chunkmistral, replace_with_chunkllama
replace_with_chunkllama(8192)

In [ ]:
# This is the full scirpt on LLAMA3
tokenizer3 = transformers.AutoTokenizer.from_pretrained(LLAMA3_path)
model3 = load_model(LLAMA3_path, tokenizer3)

prompt, answer = generate_prompt_landmark(1000 * 150, 999, 0.05)

tokens3 = tokenize(tokenizer3, prompt)
ans3 = tokenize(tokenizer3, answer)

output3 = generate(model3, tokens3, ans3.shape[-1])
decoded3 = tokenizer3.decode(output3[0][1:])

# print(prompt, answer)
# print(decoded3, answer)

# print(decoded3[:-100], answer[:-100])

model_answer3 = output3[0, -ans3.shape[-1]:].cpu()
ans_in_text = tokenizer3.decode(model_answer3)
ans_in_text = ans_in_text.replace('.', '').strip()
print(ans_in_text, answer, ans_in_text == answer)



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Quantizing model.layers blocks :   0%|          | 0/32 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]